# Deep Learning MLP Document Summary

## Overview
This document presents two comprehensive deep learning implementations using fully connected neural networks (MLPs) and CNNs for image classification tasks, incorporating various regularization techniques, optimization methods, and best practices.

## Project 1: CIFAR-10 MLP Classification

### Dataset Used
- **CIFAR-10**: 60,000 32x32 color images across 10 categories (airplane, car, bird, cat, dog, etc.)
- **Preprocessing**: Images normalized to range [0, 1] and labels converted to one-hot encoding

### Model Architecture
- **Input Layer**: Flattened 32x32x3 images into 1D vectors (3072 features)
- **Hidden Layers**: 3 fully connected (dense) layers
  - Layer 1: 512 units with ReLU activation
  - Layer 2: 256 units with ReLU activation  
  - Layer 3: 128 units with ReLU activation
- **Output Layer**: 10 units with Softmax activation for multi-class classification

### Regularization Techniques
- **L2 Regularization**: Applied to dense layers (λ = 0.001) to penalize large weights
- **Dropout**: 0.5 dropout rate after each dense layer to prevent overfitting
- **Early Stopping**: Stops training after 5 epochs without validation improvement

### Optimization Strategy
- **Primary Optimizer**: Adam with learning rate = 0.0001
- **Learning Rate Scheduler**: ReduceLROnPlateau (reduces LR by factor 0.2 when validation loss plateaus)
- **Alternative Optimizers**: SGD and RMSprop also supported

### Training Configuration
- **Epochs**: Maximum 50 epochs
- **Batch Size**: 64
- **Validation**: Uses test set for validation during training

### Callbacks Used
- **EarlyStopping**: Monitors validation loss with patience=5
- **ModelCheckpoint**: Saves best model based on validation loss
- **ReduceLROnPlateau**: Dynamic learning rate adjustment

### Performance Results
- **Test Accuracy**: 46.58%
- **Training completed**: 50 epochs with learning rate reduction at epoch 35



In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
import matplotlib.pyplot as plt

# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Normalize data to range [0, 1]
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Convert labels to one-hot encoding
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

# Define MLP model architecture
def create_mlp_model(optimizer='adam', dropout_rate=0.5, regularizer=None):
    model = models.Sequential()

    # Flatten the input image (32x32x3) to a 1D vector (3072)
    model.add(layers.Flatten(input_shape=(32, 32, 3)))

    # First dense layer
    model.add(layers.Dense(512, activation='relu', kernel_regularizer=regularizer))
    model.add(layers.Dropout(dropout_rate))  # Dropout regularization

    # Second dense layer
    model.add(layers.Dense(256, activation='relu', kernel_regularizer=regularizer))
    model.add(layers.Dropout(dropout_rate))  # Dropout regularization

    # Third dense layer
    model.add(layers.Dense(128, activation='relu', kernel_regularizer=regularizer))
    model.add(layers.Dropout(dropout_rate))  # Dropout regularization

    # Output layer with softmax activation for classification
    model.add(layers.Dense(10, activation='softmax'))

    # Compile the model with the specified optimizer
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    return model

# Set up callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
checkpoint = ModelCheckpoint('best_mlp_model.keras', monitor='val_loss', save_best_only=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)

# Choose regularization (L1, L2, L1_L2)
regularizer = regularizers.l2(0.001)

# Choose optimizer (SGD, Adam, RMSprop)
optimizer = Adam(learning_rate=0.0001)

# Create the MLP model
model = create_mlp_model(optimizer=optimizer, dropout_rate=0.5, regularizer=regularizer)

# Train the model
history = model.fit(x_train, y_train,
                    epochs=50,
                    batch_size=64,
                    validation_data=(x_test, y_test),
                    callbacks=[early_stopping, checkpoint, reduce_lr])

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print(f"Test accuracy: {test_acc:.4f}")

# Visualize the training process
# Accuracy plot
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Model Accuracy')
plt.show()

# Loss plot
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Model Loss')
plt.show()